유저, 아이템, rating의 세가지 정보있으면 기본적인 추천시스템 구현가능

#### 예제: MovieLens 데이터
    - MovieLens 100K Dataset
    - Stable benchmark dataset. 100,000 ratings from 1000 users on 1700 movies. Released 4/1998.


http://grouplens.org/datasets/movielens/100k/

In [74]:
import pandas as pd
import numpy as np

In [30]:
r_cols=['user_id','movie_id','rating','timestamp']
rating = pd.read_table('data/ml-100k/u.data',names=r_cols)
rating['date'] = pd.to_datetime(rating['timestamp'], unit='s')
rating.drop(columns='timestamp',inplace=True)
rating.head()

,user_id,movie_id,rating,date
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [26]:
u_cols = ['user_id','age','sex','job','zip_code']

user = pd.read_table('data/ml-100k/u.user',sep='|',names=u_cols,index_col='user_id')
user.head()

,age,sex,job,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [31]:
m_cols = ['movie_id','title','release_date','video_release','imdb_url']
movie = pd.read_table('data/ml-100k/u.item',encoding='ISO-8859-1',sep='|',names=m_cols,usecols=range(5),index_col='movie_id')
movie.head()

,title,release_date,video_release,imdb_url
movie_id,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [34]:
movie_rating = pd.merge(movie,rating,on='movie_id',how='inner')
movie_rating.head()

,movie_id,title,release_date,video_release,imdb_url,user_id,rating,date
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,1998-02-17 17:28:52
1,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,287,5,1997-09-27 04:21:28
2,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,148,4,1997-10-16 16:30:11
3,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,280,4,1998-04-04 14:33:46
4,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,66,3,1997-12-31 20:48:44


In [35]:
lens = pd.merge(movie_rating,user,on='user_id')

In [36]:
lens.head()

,movie_id,title,release_date,video_release,imdb_url,user_id,rating,date,age,sex,job,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,1998-02-17 17:28:52,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,1998-02-17 17:51:30,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,1998-02-17 18:20:08,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,1998-02-17 18:07:27,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,1998-02-17 17:31:36,60,M,retired,95076


In [45]:
lens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
movie_id         100000 non-null int64
title            100000 non-null object
release_date     99991 non-null object
video_release    0 non-null float64
imdb_url         99987 non-null object
user_id          100000 non-null int64
rating           100000 non-null int64
date             100000 non-null datetime64[ns]
age              100000 non-null int64
sex              100000 non-null object
job              100000 non-null object
zip_code         100000 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 9.9+ MB


In [46]:
lens.describe()

,movie_id,video_release,user_id,rating,age
count,100000.000000,0.0,100000.00000,100000.000000,100000.000000
mean,425.530130,NaN,462.48475,3.529860,32.969850
std,330.798356,NaN,266.61442,1.125674,11.562623
min,1.000000,NaN,1.00000,1.000000,7.000000
25%,175.000000,NaN,254.00000,3.000000,24.000000
50%,322.000000,NaN,447.00000,4.000000,30.000000
75%,631.000000,NaN,682.00000,4.000000,40.000000
max,1682.000000,NaN,943.00000,5.000000,73.000000


In [49]:
lens.title.value_counts().iloc[:10] # rating한 유저 수가 가장 많은 영화 top10

Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
Name: title, dtype: int64

In [68]:
lens.groupby('title').mean()[['rating']].sort_values('rating',ascending=False).iloc[:15,:] # 평점평균의  top 15

,rating
title,
They Made Me a Criminal (1939),5.000
Marlene Dietrich: Shadow and Light (1996),5.000
"Saint of Fort Washington, The (1993)",5.000
Someone Else's America (1995),5.000
Star Kid (1997),5.000
"Great Day in Harlem, A (1994)",5.000
Aiqing wansui (1994),5.000
Santa with Muscles (1996),5.000
Prefontaine (1997),5.000


In [73]:
lens.groupby('user_id').size().sort_values(ascending=False).iloc[:10] # 유저별 rating 수 top10

user_id
405    737
655    685
13     636
450    540
276    518
416    493
537    490
303    484
234    480
393    448
dtype: int64

In [77]:
movie_stats = lens.groupby('title').agg({'rating':[np.size,np.mean]}) #agg를 이용해 size와 mean 한번에 
movie_stats.head(10)

rating          
                                              size      mean
title                                                       
'Til There Was You (1997)                        9  2.333333
1-900 (1994)                                     5  2.600000
101 Dalmatians (1996)                          109  2.908257
12 Angry Men (1957)                            125  4.344000
187 (1997)                                      41  3.024390
2 Days in the Valley (1996)                     93  3.225806
20,000 Leagues Under the Sea (1954)             72  3.500000
2001: A Space Odyssey (1968)                   259  3.969112
3 Ninjas: High Noon At Mega Mountain (1998)      5  1.000000
39 Steps, The (1935)                            59  4.050847

In [78]:
atleast_100 = movie_stats['rating']['size'] >=100 # size 100 이상이라는 masking줘서 선별
movie_stats[atleast_100].sort_values(by=[('rating','mean')], ascending= False)

rating          
                                                     size      mean
title                                                              
Close Shave, A (1995)                                 112  4.491071
Schindler's List (1993)                               298  4.466443
Wrong Trousers, The (1993)                            118  4.466102
Casablanca (1942)                                     243  4.456790
Shawshank Redemption, The (1994)                      283  4.445230
Rear Window (1954)                                    209  4.387560
Usual Suspects, The (1995)                            267  4.385768
Star Wars (1977)                                      583  4.358491
12 Angry Men (1957)                                   125  4.344000
Citizen Kane (1941)                                   198  4.292929
To Kill a Mockingbird (1962)                          219  4.292237
One Flew Over the Cuckoo's Nest (1975)                264  4.291667
Silence of the Lambs, The (1991)                      390  4.289744
North by Northwest (1959)                             179  4.284916
Godfather, The (1972)                                 413  4.283293
Secrets & Lies (1996)                                 162  4.265432
Good Will Hunting (1997)                              198  4.262626
Manchurian Candidate, The (1962)                      131  4.259542
Dr. Strangelove or: How I Learned to Stop Worry...    194  4.252577
Raiders of the Lost Ark (1981)                        420  4.252381
Vertigo (1958)                                        179  4.251397
Titanic (1997)                                        350  4.245714
Lawrence of Arabia (1962)                             173  4.231214
Maltese Falcon, The (1941)                            138  4.210145
Empire Strikes Back, The (1980)                       367  4.204360
Boot, Das (1981)                                      201  4.203980
Sling Blade (1996)                                    136  4.198529
As Good As It Gets (1997)                             112  4.196429
Godfather: Part II, The (1974)                        209  4.186603
African Queen, The (1951)                             152  4.184211
...                                                   ...       ...
Ace Ventura: Pet Detective (1994)                     103  3.048544
Conan the Barbarian (1981)                            107  3.046729
Grumpier Old Men (1995)                               148  3.040541
Star Trek: The Motion Picture (1979)                  117  3.034188
Broken Arrow (1996)                                   254  3.031496
First Wives Club, The (1996)                          160  3.018750
Net, The (1995)                                       120  3.008333
Evita (1996)                                          259  2.980695
Natural Born Killers (1994)                           128  2.953125
Lost World: Jurassic Park, The (1997)                 158  2.943038
Dante's Peak (1997)                                   240  2.933333
Nutty Professor, The (1996)                           163  2.914110
101 Dalmatians (1996)                                 109  2.908257
Father of the Bride Part II (1995)                    128  2.898438
Mars Attacks! (1996)                                  217  2.847926
Multiplicity (1996)                                   134  2.835821
Alien 3 (1992)                                        100  2.830000
Volcano (1997)                                        219  2.808219
Waterworld (1995)                                     102  2.803922
Beavis and Butt-head Do America (1996)                156  2.788462
Mimic (1997)                                          101  2.742574
Down Periscope (1996)                                 101  2.702970
George of the Jungle (1997)                           162  2.685185
Batman Returns (1992)                                 142  2.683099
Batman Forever (1995)                                 114  2.666667
Spawn (1997)                           

#### Popularity Based Recommenders
예제: 음식점 데이터

In [82]:
frame = pd.read_csv('data/RS/rating_final.csv')
cuisine = pd.read_csv('data/RS/chefmozcuisine.csv')

In [83]:
frame.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [84]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [96]:
cuisine_rating = pd.merge(frame,cuisine, on='placeID')

In [97]:
cuisine_rating.head()

,userID,placeID,rating,food_rating,service_rating,Rcuisine
0,U1077,135085,2,2,2,Fast_Food
1,U1108,135085,1,2,1,Fast_Food
2,U1081,135085,1,2,1,Fast_Food
3,U1056,135085,2,2,2,Fast_Food
4,U1134,135085,2,1,2,Fast_Food


In [98]:
cuisine_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Data columns (total 6 columns):
userID            1043 non-null object
placeID           1043 non-null int64
rating            1043 non-null int64
food_rating       1043 non-null int64
service_rating    1043 non-null int64
Rcuisine          1043 non-null object
dtypes: int64(4), object(2)
memory usage: 57.0+ KB


In [103]:
cuisine_rating.groupby(['Rcuisine','placeID']).agg({'rating':[np.size,np.mean]}) # 종류별, 음식점별 rating, size와 mean

rating          
                           size      mean
Rcuisine         placeID                 
American         132583       4  1.000000
                 132851       7  1.428571
                 132872      12  0.916667
                 132951      10  1.000000
                 132958       6  1.666667
Armenian         132667       4  1.250000
Bakery           132866       5  1.400000
Bar              132921      17  1.235294
                 132937       4  1.500000
                 134983       5  1.000000
                 135019       6  1.333333
                 135026      11  1.363636
                 135041      17  1.176471
                 135048       6  1.000000
                 135052      25  1.280000
                 135057      15  1.266667
                 135059       9  1.666667
                 135069      12  0.500000
                 135071       9  1.000000
                 135073       4  1.500000
Bar_Pub_Brewery  132955       5  2.000000
                 134983       5  1.000000
                 135026      11  1.363636
                 135052      25  1.280000
                 135071       9  1.000000
                 135073       4  1.500000
Breakfast-Brunch 132954       9  1.000000
Burgers          132858       5  0.800000
                 132861       7  1.428571
                 132870       5  0.600000
...                         ...       ...
Mexican          132723      12  1.416667
                 132732       8  0.625000
                 132740       8  0.750000
                 132754      13  1.461538
                 132755       5  1.800000
                 132773       4  1.500000
                 132825      32  1.281250
                 132834      25  1.000000
                 132845       5  0.800000
                 132925       5  1.000000
                 134976       4  1.750000
                 135018       4  1.500000
                 135025      15  1.666667
                 135027       8  0.875000
                 135028      15  1.533333
                 135055       7  1.714286
                 135104       7  0.857143
                 135106      10  1.200000
Pizzeria         132733      10  1.300000
                 132869       6  1.166667
                 135043       9  0.888889
                 135053       8  1.125000
                 135058      18  1.111111
Regional         132560       4  0.500000
Seafood          135039      12  1.250000
                 135049       5  1.000000
                 135054      10  1.000000
                 135060      22  1.136364
                 135075      13  1.692308
Vietnamese       132767       6  1.166667

[112 rows x 2 columns]

geodata 추가

In [105]:
# frame = pd.read_csv('data/RS/rating_final.csv')
# cuisine = pd.read_csv('data/RS/chefmozcuisine.csv')
geodata = pd.read_csv('data/RS/geoplaces2.csv', encoding='latin1')

In [106]:
geodata.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


rating 평균이 가장 높은 place 순으로 정렬한다

In [123]:
rs_index = cuisine_rating.groupby('placeID').mean().sum(axis=1).sort_values(ascending=False).index
rs_index

Int64Index([134986, 132955, 135034, 132755, 135074, 132922, 135055, 135075,
            134999, 135018, 135025, 132768, 132958, 132667, 135035, 134976,
            132773, 135059, 132861, 135030, 132754, 132862, 135028, 132866,
            132854, 135026, 132846, 132723, 135073, 135085, 135057, 135072,
            132584, 132733, 132869, 132767, 135019, 132717, 135048, 135042,
            135052, 135106, 132825, 132613, 132937, 135044, 132875, 135060,
            132851, 132925, 135046, 135041, 135039, 132630, 135032, 135027,
            132583, 132872, 132626, 135079, 132921, 134983, 135088, 135104,
            135058, 135021, 135043, 132608, 132834, 132668, 132706, 135109,
            132572, 135054, 135053, 132856, 132858, 132951, 132954, 132740,
            132665, 135049, 135071, 132732, 132715, 132594, 132766, 132845,
            135086, 132870, 132609, 132560, 135069, 132663, 132885],
           dtype='int64', name='placeID')

In [127]:
geodata = geodata.reindex(index=rs_index)

In [130]:
geodata.drop(columns='placeID', inplace=True)

In [133]:
geodata['name'].head(10) #평점기반으로 추천했다

placeID
134986                   Restaurant Las Mananitas
132955                                  emilianos
135034                 Michiko Restaurant Japones
132755                       La Estrella de Dimas
135074          Restaurante la Parroquia Potosina
132922                       cafe punta del cielo
135055    la Cochinita Pibil Restaurante Yucateco
135075                       Mariscos El Pescador
134999                            Kiku Cuernavaca
135018                           El Oceano Dorado
Name: name, dtype: object

#### Recommendation Based on Corr

우선 cross tab이용해서 wide하게 만들어보자

In [134]:
place_crosstab = pd.pivot_table(data=frame, values='rating', index='userID',columns='placeID')
place_crosstab.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
place_crosstab = place_crosstab.fillna(0)
place_crosstab.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
U1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
place_crosstab.corr(method='pearson').head() # place기반 corr

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
placeID,,,,,,,,,,,,,,,,,,,,,
132560,1.000000,-0.018078,-0.019766,-0.035183,-0.017023,0.364754,0.397706,-0.019847,-0.018078,-0.022148,...,-0.019847,-0.030178,-0.020321,-0.060398,-0.026743,-0.022264,0.233776,-0.027060,-0.030178,-0.017023
132561,-0.018078,1.000000,-0.024298,-0.043250,-0.020925,-0.027593,-0.022222,-0.024398,-0.022222,-0.027226,...,-0.024398,-0.037097,-0.024980,-0.074246,-0.032874,-0.027369,-0.027369,-0.033264,-0.037097,-0.020925
132564,-0.019766,-0.024298,1.000000,-0.047290,-0.022880,-0.030170,-0.024298,0.218746,-0.024298,-0.029769,...,-0.026676,-0.040562,-0.027313,-0.081181,-0.035945,-0.029925,-0.029925,-0.036371,-0.040562,-0.022880
132572,-0.035183,-0.043250,-0.047290,1.000000,-0.040726,-0.053702,-0.043250,-0.047484,-0.043250,-0.052988,...,0.098133,-0.072199,-0.048617,0.077067,0.230333,-0.053267,-0.053267,0.133796,-0.021104,-0.040726
132583,-0.017023,-0.020925,-0.022880,-0.040726,1.000000,-0.025983,-0.020925,-0.022974,-0.020925,-0.025637,...,-0.022974,-0.034932,-0.023522,-0.069914,-0.030956,0.270606,-0.025772,-0.031323,-0.034932,-0.019704


In [143]:
u_corr = place_crosstab.T.corr(method='pearson')
u_corr.head() # user 기반 corr

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
userID,,,,,,,,,,,,,,,,,,,,,
U1001,1.000000,0.177522,0.106902,-0.060746,0.004865,0.050441,0.138926,-0.063104,0.048224,-0.050412,...,-0.021230,-0.021230,-0.041666,0.310872,-0.058031,0.010440,NaN,-0.066319,0.080374,-0.035629
U1002,0.177522,1.000000,0.206710,0.100876,0.035967,0.018423,0.011163,-0.070988,0.383969,-0.056710,...,-0.023882,-0.023882,-0.046871,0.358369,-0.065281,0.128066,NaN,0.271774,0.361659,0.332092
U1003,0.106902,0.206710,1.000000,-0.075665,-0.072724,0.267904,-0.076909,-0.078602,0.323740,-0.062792,...,-0.026443,-0.026443,-0.051898,0.045237,-0.072282,0.383298,NaN,-0.018685,0.424286,0.175402
U1004,-0.060746,0.100876,-0.075665,1.000000,0.115228,0.011179,0.075265,-0.065934,-0.029010,-0.052672,...,-0.022181,-0.022181,-0.043534,0.306043,-0.060633,-0.083627,NaN,0.230976,0.027992,-0.037226
U1005,0.004865,0.035967,-0.072724,0.115228,1.000000,-0.069073,0.189893,-0.063371,-0.066038,0.419462,...,-0.021319,-0.021319,-0.041842,0.023587,-0.058276,-0.080376,NaN,-0.066600,0.053809,-0.035780


In [148]:
u_corr['U1004'].sort_values(ascending=False).head() #특정 유저와 높은 corr인 유저들

userID
U1004    1.000000
U1016    0.320195
U1132    0.306043
U1024    0.285868
U1061    0.248338
Name: U1004, dtype: float64

corrwith: 두개의 DF 이용해 corr 구함

In [142]:
# similar_to_Tortas = place_crosstab.corrwith(Tortas_rating)
# corr_Tortas = pd.DataFrame(similar_to_Tortas, columns=['PearsonR'])